## Cohere hackathon

- Build a multimodal capability abstraction
- Build an example based on that

In [2]:
from dotenv import load_dotenv
from rich import print

load_dotenv(".lightning_studio/.studiorc")

import os
os.environ["LOGURU_LEVEL"] = "DEBUG"

In [3]:
import json
from agents.tools import wikipedia_search, get_current_weather, google_search, image_inspector

from agents.llms._cohere import CohereChatCompletion
from agents.tool_executor import need_tool_use

In [4]:
llm = CohereChatCompletion()

In [5]:
llm.bind_tools([get_current_weather, image_inspector, google_search, wikipedia_search])

In [6]:
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that has access to tools. Use the tools only if you don't have answers."
    },
    {"role":"user", "content":"how is the weather in London today?"}
]

output = llm.chat_completion(messages)

if need_tool_use(output):
    fname = output.choices[0].message.tool_calls[0].function.name
    print(f"Calling `{fname}` API")
    tool_results = llm.run_tool(output)
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [
        {"role": "user", "content": "Please answer my question based on the above context."}
        ]
    output = llm.chat_completion(updated_messages)

print(output.choices[0].message.content)

2024-05-09 21:33:13.367 | DEBUG    | agents.llms._cohere:chat_completion:58 - content='' additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'd86dc590-5c82-41e0-b19c-6fa0dc4251a7', 'tool_calls': [{'id': '1a07b7a389e6451297e1c6f625f694d2', 'function': {'name': 'get_current_weather', 'arguments': '{"city": "London"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 10}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'd86dc590-5c82-41e0-b19c-6fa0dc4251a7', 'tool_calls': [{'id': '1a07b7a389e6451297e1c6f625f694d2', 'function': {'name': 'get_current_weather', 'arguments': '{"city": "London"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 10}} id='run-476d1c0c-7b96-4ee2-afd7-9fc5c4357882-0' tool_calls=[{'name': 'get_current_weather', 'args': {'city': 'London'}, 'id': '55a96713

Calling `get_current_weather` API

2024-05-09 21:33:13.372 | DEBUG    | agents.tool_executor:call_tool:88 - Function get_current_weather invoked with {'city': 'London'}
2024-05-09 21:33:14.066 | DEBUG    | agents.tool_executor:call_tool:90 - Function get_current_weather, responded with {"FeelsLikeC": "16", "FeelsLikeF": "61", "cloudcover": "0", "humidity": "77", "localObsDateTime": "2024-05-09 09:15 PM", "observation_time": "08:15 PM", "precipInches": "0.0", "precipMM": "0.0", "pressure": "1025", "pressureInches": "30", "temp_C": "16", "temp_F": "61", "uvIndex": "1", "visibility": "10", "visibilityMiles": "6", "weatherCode": "113", "weatherDesc": [{"value": "Clear"}], "weatherIconUrl": [{"value": ""}], "winddir16Point": "SSW", "winddirDegree": "200", "windspeedKmph": "15", "windspeedMiles": "9"}
2024-05-09 21:33:18.168 | DEBUG    | agents.llms._cohere:chat_completion:58 - content="The weather in London, UK, as of 8:15 PM local time, is clear with a temperature of 16 degrees Celsius or 61 degrees Fahrenheit. The humidity

The weather in London, UK, as of 8:15 PM local time, is clear with a temperature of 16 degrees Celsius or 61 
degrees Fahrenheit. The humidity is at 77%, and there is no precipitation. The UV index is low at 1, and the wind 
is blowing from the SSW direction at 9 miles per hour. Visibility is good at 6 miles, and the pressure is at 1025 
millibars. So, overall, it's a pleasant evening in London with comfortable temperatures and clear skies.

In [7]:
image_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

messages = [
    {"role": "system", "content": "You are a helpful assistant that has access to tools and use that to help humans."},
    {"role": "user", "content": f"Suggest me a location where I can go in London which looks similar to this image ```{image_path}```"}
]
output = llm.chat_completion(messages)

2024-05-09 21:33:24.760 | DEBUG    | agents.llms._cohere:chat_completion:58 - content='' additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '71b2187d-767d-4e4c-9d48-b96ab4f9dc73', 'tool_calls': [{'id': '7da69f56ee1643bebf56be3a892a5054', 'function': {'name': 'image_inspector', 'arguments': '{"image_url_or_path": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"}'}, 'type': 'function'}, {'id': '65abe7d29b5c44f783454c330674f042', 'function': {'name': 'google_search', 'arguments': '{"query": "London nature reserve with boardwalk"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 85}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '71b2187d-767d-4e4c-9d48-b96ab4f9dc73', 'to

In [ ]:
print(output)

In [8]:
tool_results = llm.run_tool(output)

2024-05-09 21:33:45.204 | DEBUG    | agents.tool_executor:call_tool:88 - Function image_inspector invoked with {'image_url_or_path': 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg'}
2024-05-09 21:33:49.840 | DEBUG    | agents.tool_executor:call_tool:90 - Function image_inspector, responded with {
  "image_url_or_path": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
  "image description": " The image depicts a long wooden boardwalk cutting through a lush green field. The boardwalk is situated on the right side of the frame, stretching across the scene from one end to another. The field surrounding the boardwalk is filled with tall grass and wildflowers, creating a picturesque landscape. The sky above is blue with scattered clouds, adding depth to the overall scen

In [10]:
if need_tool_use(output):
    tool_results = [llm.run_tool(output)[0]]  # use single tool
    tool_results[0]["role"] = "assistant"

    updated_messages = messages + tool_results
    updated_messages = updated_messages + [
        {
            "role": "user", "content": "Please answer my question based on the above context."
        }
        ]
    output = llm.chat_completion(updated_messages)

2024-05-09 21:34:39.151 | DEBUG    | agents.tool_executor:call_tool:88 - Function image_inspector invoked with {'image_url_or_path': 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg'}
2024-05-09 21:34:40.446 | DEBUG    | agents.tool_executor:call_tool:90 - Function image_inspector, responded with {
  "image_url_or_path": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
  "image description": " The image depicts a wooden boardwalk with a grassy field extending into the distance. The boardwalk is situated on a small island, surrounded by water and offering a scenic view of the landscape. The sky above appears to be cloudy, creating a slightly overcast atmosphere."
}
2024-05-09 21:34:40.447 | DEBUG    | agents.tool_executor:call_tool:88 - Function google_search invoked

In [15]:
updated_messages = messages + tool_results
updated_messages = updated_messages + [
    {
        "role": "user", "content": "Based on the image description suggest me a good place to walk in london"
    }
    ]
output = llm.chat_completion(updated_messages)
print(output)

2024-05-09 21:37:29.236 | DEBUG    | agents.llms._cohere:chat_completion:58 - content='' additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '5e21d550-5c70-4935-a924-15df1e2b59d8', 'tool_calls': [{'id': '5301113c2e6744b5bfebf1323d0574a6', 'function': {'name': 'google_search', 'arguments': '{"query": "London walk similar to wooden boardwalk and grassy field"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 16}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '5e21d550-5c70-4935-a924-15df1e2b59d8', 'tool_calls': [{'id': '5301113c2e6744b5bfebf1323d0574a6', 'function': {'name': 'google_search', 'arguments': '{"query": "London walk similar to wooden boardwalk and grassy field"}'}, 'type': 'function'}], 'token_count': {'output_tokens': 16}} id='run-907b27ca-22bd-4efb-b9a9-daaa502cd048-

ChatCompletion(
    id='run-907b27ca-22bd-4efb-b9a9-daaa502cd048-0',
    object='',
    created=1715290649,
    model='Cohere',
    choices=[
        Choice(
            index=0,
            logprobs=None,
            message=Message(
                role='assistant',
                content='',
                tool_calls=[
                    ToolCall(
                        id='5301113c2e6744b5bfebf1323d0574a6',
                        type='function',
                        function=FunctionInfo(
                            name='google_search',
                            arguments='{"query": "London walk similar to wooden boardwalk and grassy field"}'
                        )
                    )
                ]
            ),
            finish_reason='stop'
        )
    ],
    usage=Usage(prompt_tokens=-1, completion_tokens=16, total_tokens=-1)
)